In [2]:
import os
import numpy as np 
import pandas as pd

import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input

In [3]:
df_train = pd.read_csv("data/type_encodings.csv")
print(df_train.shape)
df_train.head()

(8430, 19)


,file,type_Bug,type_Dark,type_Dragon,type_Electric,type_Fairy,type_Fighting,type_Fire,type_Flying,type_Ghost,type_Grass,type_Ground,type_Ice,type_Normal,type_Poison,type_Psychic,type_Rock,type_Steel,type_Water
0,0001.png,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0002.png,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0003.png,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0004.png,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0005.png,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [4]:
image_size = 450
batch_size = 32

In [5]:
def build_decoder(with_labels=True, target_size=(image_size, image_size)):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        image = tf.image.decode_png(file_bytes, channels=3)
        image = tf.cast(image, tf.float32) / 255.0
        image = tf.image.resize(image, target_size)
        image = preprocess_input(image)
        
        return image
    
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    
    return decode_with_labels if with_labels else decode

In [6]:
def build_augmenter(with_labels=True):
    def augment(image):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_saturation(image, 0.8, 1.2)
        image = tf.image.random_brightness(image, 0.2)
        image = tf.image.random_contrast(image, 0.8, 1.2)
        image = tf.image.random_hue(image, 0.2)
        
        return image
    

    def augment_with_labels(image, label):
        return augment(image), label
    
    
    return augment_with_labels if with_labels else augment

In [7]:
def build_dataset(paths, labels=None, bsize=32, decode_function=None, augment_function=None, augment=True):
    if decode_function is None:
        decode_function = build_decoder(labels is not None)

    if augment_function is None:
        augment_function = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dataset = tf.data.Dataset.from_tensor_slices(slices)  
    dataset = dataset.map(decode_function, num_parallel_calls=AUTO)
    dataset = dataset.map(augment_function, num_parallel_calls=AUTO) if augment else dataset

    dataset = dataset.batch(bsize).prefetch(AUTO) 

    return dataset

In [8]:
model = tf.keras.models.load_model('model.h5')

In [9]:
load_dir = "data/images/test/"

test_paths = [os.path.join(load_dir, path) for path in os.listdir(load_dir)]

test_decoder = build_decoder(with_labels=False, target_size=(image_size, image_size))

data_test = build_dataset(test_paths, bsize=batch_size, augment=False, decode_function=test_decoder)

In [10]:
y_preds = model.predict(data_test, verbose=1)

5/5 [==============================] - 46s 9s/step


In [35]:
df_test = pd.DataFrame([os.listdir(load_dir)] + [y_pred for y_pred in list(zip(*y_preds))], df_train.columns).T

print(df_test.shape)
df_test.head(9)

(151, 19)


,file,type_Bug,type_Dark,type_Dragon,type_Electric,type_Fairy,type_Fighting,type_Fire,type_Flying,type_Ghost,type_Grass,type_Ground,type_Ice,type_Normal,type_Poison,type_Psychic,type_Rock,type_Steel,type_Water
0,0001.png,0.000264,0.000096,0.000059,0.000045,0.000045,0.000061,0.000027,0.000038,0.000549,0.999961,0.000034,0.000135,0.000032,0.999417,0.000051,0.000131,0.000018,0.000030
1,0002.png,0.000267,0.000376,0.000034,0.000149,0.000025,0.000065,0.000031,0.000029,0.000473,0.999980,0.000015,0.000270,0.000015,0.988147,0.000161,0.000066,0.000014,0.000030
2,0003.png,0.000196,0.000040,0.000035,0.000012,0.000063,0.000021,0.000005,0.000029,0.000076,0.999998,0.000004,0.000143,0.000003,0.997504,0.000060,0.000010,0.000010,0.000002
3,0004.png,0.000005,0.000204,0.000014,0.000025,0.000003,0.001325,0.999938,0.000029,0.000002,0.000014,0.000022,0.000102,0.000002,0.000006,0.003179,0.000002,0.000001,0.000364
4,0005.png,0.000027,0.002298,0.000224,0.000469,0.000025,0.000643,0.999985,0.001148,0.000009,0.000011,0.001242,0.000072,0.000042,0.000003,0.000207,0.000042,0.000010,0.000021
5,0006.png,0.000001,0.000517,0.001545,0.000003,0.000008,0.000289,0.999997,0.989674,0.000001,0.000002,0.000108,0.000071,0.000006,0.000003,0.000074,0.000137,0.000002,0.000003
6,0007.png,0.000070,0.000006,0.000010,0.000000,0.000002,0.000588,0.000006,0.000031,0.000001,0.000020,0.000076,0.000071,0.000012,0.000365,0.000027,0.000161,0.000001,0.999984
7,0008.png,0.003601,0.000016,0.000051,0.000075,0.000080,0.000074,0.000004,0.000298,0.000006,0.000022,0.000128,0.000021,0.000053,0.000053,0.000025,0.000772,0.000045,0.999944
8,0009.png,0.000137,0.000026,0.000072,0.000003,0.000024,0.000251,0.000005,0.000087,0.000006,0.000045,0.005040,0.001137,0.000193,0.000016,0.000369,0.476890,0.000074,0.998082


In [36]:
type_dict = df_test.to_dict(orient='list')

type_dict['predicted_types'] = []

for i in range(len(type_dict['file'])):
    types = [key for key in type_dict.keys() if key not in ['file', 'predicted_types']]
    type_dict['predicted_types'].append([tpe[5:] for tpe in types if type_dict[tpe][i] > 0.66])
    
type_dict['predicted_types'] = [",".join(row) for row in type_dict['predicted_types']]

df_test = pd.DataFrame.from_dict(type_dict)

cols = df_test.columns.tolist()
cols = [cols[0]] + [cols[-1]] + cols[1:-1]

df_test = df_test[cols]

print(df_test.shape)
df_test.head(9)

(151, 20)


,file,predicted_types,type_Bug,type_Dark,type_Dragon,type_Electric,type_Fairy,type_Fighting,type_Fire,type_Flying,type_Ghost,type_Grass,type_Ground,type_Ice,type_Normal,type_Poison,type_Psychic,type_Rock,type_Steel,type_Water
0,0001.png,"Grass,Poison",0.000264,0.000096,0.000059,4.536115e-05,0.000045,0.000061,0.000027,0.000038,5.493760e-04,0.999961,0.000034,0.000135,0.000032,0.999417,0.000051,0.000131,1.793130e-05,0.000030
1,0002.png,"Grass,Poison",0.000267,0.000376,0.000034,1.489222e-04,0.000025,0.000065,0.000031,0.000029,4.727244e-04,0.999980,0.000015,0.000270,0.000015,0.988147,0.000161,0.000066,1.441740e-05,0.000030
2,0003.png,"Grass,Poison",0.000196,0.000040,0.000035,1.194831e-05,0.000063,0.000021,0.000005,0.000029,7.607859e-05,0.999998,0.000004,0.000143,0.000003,0.997504,0.000060,0.000010,9.729082e-06,0.000002
3,0004.png,Fire,0.000005,0.000204,0.000014,2.524184e-05,0.000003,0.001325,0.999938,0.000029,2.018253e-06,0.000014,0.000022,0.000102,0.000002,0.000006,0.003179,0.000002,7.130916e-07,0.000364
4,0005.png,Fire,0.000027,0.002298,0.000224,4.691482e-04,0.000025,0.000643,0.999985,0.001148,9.429445e-06,0.000011,0.001242,0.000072,0.000042,0.000003,0.000207,0.000042,1.026122e-05,0.000021
5,0006.png,"Fire,Flying",0.000001,0.000517,0.001545,3.311720e-06,0.000008,0.000289,0.999997,0.989674,1.208868e-06,0.000002,0.000108,0.000071,0.000006,0.000003,0.000074,0.000137,1.621698e-06,0.000003
6,0007.png,Water,0.000070,0.000006,0.000010,2.593269e-07,0.000002,0.000588,0.000006,0.000031,5.730649e-07,0.000020,0.000076,0.000071,0.000012,0.000365,0.000027,0.000161,5.019169e-07,0.999984
7,0008.png,Water,0.003601,0.000016,0.000051,7.499216e-05,0.000080,0.000074,0.000004,0.000298,5.753751e-06,0.000022,0.000128,0.000021,0.000053,0.000053,0.000025,0.000772,4.495028e-05,0.999944
8,0009.png,Water,0.000137,0.000026,0.000072,3.092974e-06,0.000024,0.000251,0.000005,0.000087,6.003422e-06,0.000045,0.005040,0.001137,0.000193,0.000016,0.000369,0.476890,7.369825e-05,0.998082


In [32]:
df_test.to_csv("data/predictions.csv", index=False)